# Word Count


In [0]:
%fs head /FileStore/tables/july2023data/sample.txt

hello world
how are you
hello world
I am doing fine
hello again
hope you are fine
hello world
hello world
I am also fine

In [0]:
df = (spark
      .read
      .format("text")
      .load("/FileStore/tables/july2023data/sample.txt")
      .toDF("lines"))

display(df)

lines
hello world
how are you
hello world
I am doing fine
hello again
hope you are fine
hello world
hello world
I am also fine


In [0]:
from pyspark.sql.functions import split
split_df = df.select(split("lines"," ").alias("words"))

display(split_df)

words
"List(hello, world)"
"List(how, are, you)"
"List(hello, world)"
"List(I, am, doing, fine)"
"List(hello, again)"
"List(hope, you, are, fine)"
"List(hello, world)"
"List(hello, world)"
"List(I, am, also, fine)"


In [0]:
from pyspark.sql.functions import explode

explode_df = split_df.select(explode("words").alias("word"))
display(explode_df)

word
hello
world
how
are
you
hello
world
I
am
doing


In [0]:
agg_df = explode_df.groupby("word").count()
display(agg_df)

word,count
hope,1
you,2
fine,3
how,1
hello,5
again,1
doing,1
are,2
world,4
I,2


In [0]:
from pyspark.sql.functions import desc
sorted_df = agg_df.orderBy(desc("count"))
sorted_df.show(5)

+-----+-----+
| word|count|
+-----+-----+
|hello|    5|
|world|    4|
| fine|    3|
|  you|    2|
|  are|    2|
+-----+-----+
only showing top 5 rows



In [0]:
%fs head  /FileStore/tables/july2023data/input.json

{
"name": "xyz",
"values": ["a", "b", "c"]
}

In [0]:
input_df = (spark
            .read
            .format("json")
            .option("multiline","true")
            .load("/FileStore/tables/july2023data/input.json"))

In [0]:
input_df.show()

+----+---------+
|name|   values|
+----+---------+
| xyz|[a, b, c]|
+----+---------+



In [0]:
from pyspark.sql.functions import explode
explode_df = input_df.select("name", explode("values").alias("value"))
display(explode_df)

name,value
xyz,a
xyz,b
xyz,c


In [0]:
%fs head  /FileStore/tables/students_nested_json_data.json

[
 {
 "name":"Ram",
 "Education":[
 {
 "Qualification":"BE",
 "year":2011
 },
 {
 "Qualification":"ME",
 "year":2013
 }
 ]
 },
 {
 "name":"Rohit",
 "Education":[
 {
 "Qualification":"BE",
 "year":2010
 }
 ]
 }
]

In [0]:
sn_df = (spark.read.format("json")
         .option("multiline","true")
         .load("/FileStore/tables/students_nested_json_data.json"))

In [0]:
sn_df.show()

+--------------------+-----+
|           Education| name|
+--------------------+-----+
|[{BE, 2011}, {ME,...|  Ram|
|        [{BE, 2010}]|Rohit|
+--------------------+-----+



In [0]:
sn_df.printSchema()

root
 |-- Education: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- Qualification: string (nullable = true)
 |    |    |-- year: long (nullable = true)
 |-- name: string (nullable = true)



In [0]:
explode_sn_df = sn_df.select("name",explode("Education").alias("education"))
explode_sn_df.show()

+-----+----------+
| name| education|
+-----+----------+
|  Ram|{BE, 2011}|
|  Ram|{ME, 2013}|
|Rohit|{BE, 2010}|
+-----+----------+



In [0]:
explode_sn_df.printSchema()

root
 |-- name: string (nullable = true)
 |-- education: struct (nullable = true)
 |    |-- Qualification: string (nullable = true)
 |    |-- year: long (nullable = true)



In [0]:
final_df = explode_sn_df.select("name","education.Qualification","education.year")
final_df.show()

+-----+-------------+----+
| name|Qualification|year|
+-----+-------------+----+
|  Ram|           BE|2011|
|  Ram|           ME|2013|
|Rohit|           BE|2010|
+-----+-------------+----+



In [0]:
final_df_2 = explode_sn_df.select("name","education.*")
final_df_2.show()

+-----+-------------+----+
| name|Qualification|year|
+-----+-------------+----+
|  Ram|           BE|2011|
|  Ram|           ME|2013|
|Rohit|           BE|2010|
+-----+-------------+----+



#Infer Schema

In [0]:
%fs head /FileStore/tables/employee.csv

ID,Name,HomeTown,Salary,JoiningDate
1,Arpit,Burhanpur,50000.00,2018-11-14
2,Benu,Bhubaneswar,100000.00,2018-11-19
3,Dilsher,Amritsar,100000.00,2018-11-19
4,Kiran,Bengaluru,50000.00,2018-11-15

In [0]:
emp_csv = (spark
           .read
           .format("csv")
           .option("header","true")
           .option("inferSchema","true")
           .load("/FileStore/tables/employee.csv"))
display(emp_csv)

# infer schema adds one more spark job not a good practice as it slows down the process

ID,Name,HomeTown,Salary,JoiningDate
1,Arpit,Burhanpur,50000.0,2018-11-14
2,Benu,Bhubaneswar,100000.0,2018-11-19
3,Dilsher,Amritsar,100000.0,2018-11-19
4,Kiran,Bengaluru,50000.0,2018-11-15


In [0]:
emp_csv.printSchema()
# ideally the default data type is string type

root
 |-- ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- HomeTown: string (nullable = true)
 |-- Salary: double (nullable = true)
 |-- JoiningDate: date (nullable = true)



# user defined schema/ custom schema usind Struct

In [0]:
%fs head /FileStore/tables/july2023data/department.csv

EmployeeID,DepartmentName,Client,OnboardedDate
1,Data Engineering,Funding Circle,2019-01-15
2,Data Engineering,Funding Circle,2018-11-19
3,Data Analytics,Funding Circle,2018-11-19
4,Data Analytics,Funding Circle,2018-12-16

In [0]:
# Strurt type defines a row/ record
#struct field defines a field/column
from pyspark.sql.types import StructType, IntegerType, StructField, StringType, DateType

dept_schema = StructType([StructField("EmployeeID",IntegerType()),
                          StructField("DepartmentName",StringType()),
                           StructField("Client",StringType()),
                           StructField("OnboardedDate",DateType())])

In [0]:
dept_df = (spark.read
           .format("csv")
           .option("header","true")
           .schema(dept_schema)
           .load("/FileStore/tables/july2023data/department.csv"))
dept_df.show()

+----------+----------------+--------------+-------------+
|EmployeeID|  DepartmentName|        Client|OnboardedDate|
+----------+----------------+--------------+-------------+
|         1|Data Engineering|Funding Circle|   2019-01-15|
|         2|Data Engineering|Funding Circle|   2018-11-19|
|         3|  Data Analytics|Funding Circle|   2018-11-19|
|         4|  Data Analytics|Funding Circle|   2018-12-16|
+----------+----------------+--------------+-------------+



In [0]:
dept_df.printSchema()

root
 |-- EmployeeID: integer (nullable = true)
 |-- DepartmentName: string (nullable = true)
 |-- Client: string (nullable = true)
 |-- OnboardedDate: date (nullable = true)



# Custom/ User defined schema using DDL String method

In [0]:
%fs head /FileStore/tables/part_00000-1

[Truncated to first 65536 bytes]
1,2013-07-25 00:00:00.0,11599,CLOSED
2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT
3,2013-07-25 00:00:00.0,12111,COMPLETE
4,2013-07-25 00:00:00.0,8827,CLOSED
5,2013-07-25 00:00:00.0,11318,COMPLETE
6,2013-07-25 00:00:00.0,7130,COMPLETE
7,2013-07-25 00:00:00.0,4530,COMPLETE
8,2013-07-25 00:00:00.0,2911,PROCESSING
9,2013-07-25 00:00:00.0,5657,PENDING_PAYMENT
10,2013-07-25 00:00:00.0,5648,PENDING_PAYMENT
11,2013-07-25 00:00:00.0,918,PAYMENT_REVIEW
12,2013-07-25 00:00:00.0,1837,CLOSED
13,2013-07-25 00:00:00.0,9149,PENDING_PAYMENT
14,2013-07-25 00:00:00.0,9842,PROCESSING
15,2013-07-25 00:00:00.0,2568,COMPLETE
16,2013-07-25 00:00:00.0,7276,PENDING_PAYMENT
17,2013-07-25 00:00:00.0,2667,COMPLETE
18,2013-07-25 00:00:00.0,1205,CLOSED
19,2013-07-25 00:00:00.0,9488,PENDING_PAYMENT
20,2013-07-25 00:00:00.0,9198,PROCESSING
21,2013-07-25 00:00:00.0,2711,PENDING
22,2013-07-25 00:00:00.0,333,COMPLETE
23,2013-07-25 00:00:00.0,4367,PENDING_PAYMENT
24,2013-07-25 00:00:00.0,11441,CLOSED
25,2013-07-25 00:00:00.0,9503,CLOSED
26,2013-07-25 00:00:00.0,7562,COMPLETE
27,2013-07-25 00:00:00.0,3241,PENDING_PAYMENT
28,2013-07-25 00:00:00.0,656,COMPLETE
29,2013-07-25 00:00:00.0,196,PROCESSING
30,2013-07-25 00:00:00.0,10039,PENDING_PAYMENT
31,2013-07-25 00:00:00.0,6983,PAYMENT_REVIEW
32,2013-07-25 00:00:00.0,3960,COMPLETE
33,2013-07-25 00:00:00.0,5793,PENDING_PAYMENT
34,2013-07-25 00:00:00.0,4189,PROCESSING
35,2013-07-25 00:00:00.0,4840,COMPLETE
36,2013-07-25 00:00:00.0,5649,PENDING
37,2013-07-25 00:00:00.0,5863,CLOSED
38,2013-07-25 00:00:00.0,11586,PROCESSING
39,2013-07-25 00:00:00.0,8214,PENDING
40,2013-07-25 00:00:00.0,12092,PENDING_PAYMENT
41,2013-07-25 00:00:00.0,8136,PENDING_PAYMENT
42,2013-07-25 00:00:00.0,9776,PENDING
43,2013-07-25 00:00:00.0,7776,PENDING_PAYMENT
44,2013-07-25 00:00:00.0,10500,PENDING
45,2013-07-25 00:00:00.0,2636,COMPLETE
46,2013-07-25 00:00:00.0,1549,ON_HOLD
47,2013-07-25 00:00:00.0,8487,PENDING_PAYMENT
48,2013-07-25 00:00:00.0,12186,PROCESSING
49,2013-07-25 00:00:00.0,1871,PENDING
50,2013-07-25 00:00:00.0,5225,CANCELED
51,2013-07-25 00:00:00.0,12271,CLOSED
52,2013-07-25 00:00:00.0,5126,PENDING_PAYMENT
53,2013-07-25 00:00:00.0,4701,PROCESSING
54,2013-07-25 00:00:00.0,10628,PENDING_PAYMENT
55,2013-07-25 00:00:00.0,2052,PENDING
56,2013-07-25 00:00:00.0,10519,COMPLETE
57,2013-07-25 00:00:00.0,7073,CLOSED
58,2013-07-25 00:00:00.0,9213,PENDING_PAYMENT
59,2013-07-25 00:00:00.0,11644,PENDING_PAYMENT
60,2013-07-25 00:00:00.0,8365,PENDING_PAYMENT
61,2013-07-25 00:00:00.0,4791,CLOSED
62,2013-07-25 00:00:00.0,9111,CLOSED
63,2013-07-25 00:00:00.0,1148,COMPLETE
64,2013-07-25 00:00:00.0,5579,PENDING_PAYMENT
65,2013-07-25 00:00:00.0,5903,COMPLETE
66,2013-07-25 00:00:00.0,1792,ON_HOLD
67,2013-07-25 00:00:00.0,1406,COMPLETE
68,2013-07-25 00:00:00.0,4320,PENDING
69,2013-07-25 00:00:00.0,2821,SUSPECTED_FRAUD
70,2013-07-25 00:00:00.0,11809,PENDING_PAYMENT
71,2013-07-25 00:00:00.0,8646,COMPLETE
72,2013-07-25 00:00:00.0,4349,COMPLETE
73,2013-07-25 00:00:00.0,8504,PENDING_PAYMENT
74,2013-07-25 00:00:00.0,662,PENDING_PAYMENT
75,2013-07-25 00:00:00.0,2505,PENDING_PAYMENT
76,2013-07-25 00:00:00.0,6898,COMPLETE
77,2013-07-25 00:00:00.0,7915,PENDING_PAYMENT
78,2013-07-25 00:00:00.0,8619,PAYMENT_REVIEW
79,2013-07-25 00:00:00.0,7327,PENDING_PAYMENT
80,2013-07-25 00:00:00.0,3007,COMPLETE
81,2013-07-25 00:00:00.0,674,PROCESSING
82,2013-07-25 00:00:00.0,3566,PENDING_PAYMENT
83,2013-07-25 00:00:00.0,1265,COMPLETE
84,2013-07-25 00:00:00.0,6789,PROCESSING
85,2013-07-25 00:00:00.0,1485,PENDING
86,2013-07-25 00:00:00.0,6680,PENDING_PAYMENT
87,2013-07-25 00:00:00.0,3065,CLOSED
88,2013-07-25 00:00:00.0,3809,COMPLETE
89,2013-07-25 00:00:00.0,824,ON_HOLD
90,2013-07-25 00:00:00.0,9131,CLOSED
91,2013-07-25 00:00:00.0,8912,COMPLETE
92,2013-07-25 00:00:00.0,6932,COMPLETE
93,2013-07-25 00:00:00.0,2256,PENDING_PAYMENT
94,2013-07-25 00:00:00.0,11589,PROCESSING
95,2013-07-25 00:00:00.0,9032,COMPLETE
96,2013-07-25 00:00:00.0,8683,PENDING
97,2013-07-25 

In [0]:
order_schema = "order_id int, order_date date, order_customer_id int, order_status string"

In [0]:
orders_df = (spark.read
             .format("csv")
             .schema(order_schema)
             .load("/FileStore/tables/part_00000-1")
             
             .toDF("order_id","order_date","order_customer_id","order_status"))

orders_df.show()

+--------+----------+-----------------+---------------+
|order_id|order_date|order_customer_id|   order_status|
+--------+----------+-----------------+---------------+
|       1|2013-07-25|            11599|         CLOSED|
|       2|2013-07-25|              256|PENDING_PAYMENT|
|       3|2013-07-25|            12111|       COMPLETE|
|       4|2013-07-25|             8827|         CLOSED|
|       5|2013-07-25|            11318|       COMPLETE|
|       6|2013-07-25|             7130|       COMPLETE|
|       7|2013-07-25|             4530|       COMPLETE|
|       8|2013-07-25|             2911|     PROCESSING|
|       9|2013-07-25|             5657|PENDING_PAYMENT|
|      10|2013-07-25|             5648|PENDING_PAYMENT|
|      11|2013-07-25|              918| PAYMENT_REVIEW|
|      12|2013-07-25|             1837|         CLOSED|
|      13|2013-07-25|             9149|PENDING_PAYMENT|
|      14|2013-07-25|             9842|     PROCESSING|
|      15|2013-07-25|             2568|       CO

In [0]:
orders_df.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- order_date: date (nullable = true)
 |-- order_customer_id: integer (nullable = true)
 |-- order_status: string (nullable = true)

